In [ ]:
%matplotlib inline

# MDF-based DOE on the Sobieski SSBJ test case.


In [ ]:
from __future__ import annotations

from os import name as os_name

from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import generate_n2_plot
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace
from gemseo.settings.doe import PYDOE_LHS_Settings

## Instantiate the  disciplines

First, we instantiate the four disciplines of the use case:
[SobieskiPropulsion][gemseo.problems.mdo.sobieski.disciplines.SobieskiPropulsion],
[SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics],
[SobieskiMission][gemseo.problems.mdo.sobieski.disciplines.SobieskiMission]
and [SobieskiStructure][gemseo.problems.mdo.sobieski.disciplines.SobieskiStructure].



In [ ]:
disciplines = create_discipline([
    "SobieskiPropulsion",
    "SobieskiAerodynamics",
    "SobieskiMission",
    "SobieskiStructure",
])

We can quickly access the most relevant information of any discipline (name, inputs,
and outputs) with Python's `print()` function. Moreover, we can get the default
input values of a discipline with the attribute [Discipline.default_input_data][gemseo.core.discipline.discipline.Discipline.default_input_data].



In [ ]:
for discipline in disciplines:
    print(discipline)  # noqa: T201
    print(f"Default inputs: {discipline.default_input_data}")  # noqa: T201pass

You may also be interested in plotting the couplings of your disciplines.
A quick way of getting this information is the high-level function
[generate_n2_plot()][gemseo.generate_n2_plot]. A much more detailed explanation of coupling
visualization is available [here][coupling-visualization].



In [ ]:
generate_n2_plot(disciplines, save=False, show=True)

## Build, execute and post-process the scenario
Then, we build the scenario which links the disciplines
with the formulation and the optimization algorithm. Here, we use the
[BiLevel][gemseo.formulations.bilevel.BiLevel] formulation. We tell the scenario to minimize -y_4
instead of minimizing y_4 (range), which is the default option.

We need to define the design space.



In [ ]:
design_space = SobieskiDesignSpace()
design_space

### Instantiate the scenario



In [ ]:
scenario = create_scenario(
    disciplines,
    "y_4",
    design_space,
    maximize_objective=True,
    scenario_type="DOE",
    formulation_name="MDF",
)

### Set the design constraints



In [ ]:
for constraint in ["g_1", "g_2", "g_3"]:
    scenario.add_constraint(constraint, constraint_type="ineq")

### Visualize the XDSM

Generate the XDSM on the fly:

- `log_workflow_status=True` will log the status of the workflow in the console,
- `save_html` (default `True`) will generate a self-contained HTML file,
  that can be automatically opened using `show_html=True`.



In [ ]:
scenario.xdsmize(save_html=False)

### Execute the scenario

Use provided analytic derivatives



In [ ]:
scenario.set_differentiation_method()

### Multiprocessing

It is possible to run a DOE in parallel using multiprocessing, in order to do
this, we specify the number of processes to be used for the computation of
the samples.



!!! warning
      The multiprocessing option has some limitations on Windows.
      Due to problems with building the documentation, we disable it in this example.
      The features [MemoryFullCache][gemseo.caches.memory_full_cache.MemoryFullCache] and [HDF5Cache][gemseo.caches.hdf5_cache.HDF5Cache] are not
      available for multiprocessing on Windows.
      As an alternative, we recommend the method
      [set_optimization_history_backup()][gemseo.scenarios.doe_scenario.DOEScenario.set_optimization_history_backup].



We define the algorithm settings. Here the criterion "center" of pyDOE
centers the points within the sampling intervals.
Note that it is also possible to pass the settings one by one, see
[this page][algorithm-settings].



In [ ]:
lhs_settings = PYDOE_LHS_Settings(
    n_samples=30,
    criterion="center",
    # Evaluate gradient of the MDA
    # with coupled adjoint
    eval_jac=True,
    # Run in parallel on 1 or 4 processors
    n_processes=1 if os_name == "nt" else 4,
)

scenario.execute(lhs_settings)

!!! warning
      On Windows, the progress bar may show duplicated instances during the
      initialization of each subprocess. In some cases it may also print the
      conclusion of an iteration ahead of another one that was concluded first.
      This is a consequence of the pickling process and does not affect the
      computations of the scenario.



### Exporting the problem data.
After the execution of the scenario, you may want to export your data to use it
elsewhere. The method [to_dataset()][gemseo.scenarios.base_scenario.BaseScenario.to_dataset] will allow you to export
your results to a [Dataset][gemseo.datasets.dataset.Dataset], the basic GEMSEO class to store data.



In [ ]:
dataset = scenario.to_dataset("a_name_for_my_dataset")

### Plot the optimization history view



In [ ]:
scenario.post_process(post_name="OptHistoryView", save=False, show=True)

!!! tip

      Each post-processing method requires different inputs and offers a variety
      of customization options. Use the high-level function
      [get_post_processing_options_schema()][gemseo.get_post_processing_options_schema] to print a table with
      the attributes for any post-processing algo. Or refer to our dedicated page:
      :ref:`gen_post_algos`.



### Plot the scatter matrix



In [ ]:
scenario.post_process(
    post_name="ScatterPlotMatrix",
    save=False,
    show=True,
    variable_names=["y_4", "x_shared"],
)

### Plot correlations



In [ ]:
scenario.post_process(post_name="Correlations", save=False, show=True)